In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.metrics import precision_score, recall_score

In [2]:
path = "./"
train = pd.read_csv(path + "input/pfm_train.csv")
test = pd.read_csv(path + "input/pfm_test.csv")
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1100 non-null   int64 
 1   Attrition                 1100 non-null   int64 
 2   BusinessTravel            1100 non-null   object
 3   Department                1100 non-null   object
 4   DistanceFromHome          1100 non-null   int64 
 5   Education                 1100 non-null   int64 
 6   EducationField            1100 non-null   object
 7   EmployeeNumber            1100 non-null   int64 
 8   EnvironmentSatisfaction   1100 non-null   int64 
 9   Gender                    1100 non-null   object
 10  JobInvolvement            1100 non-null   int64 
 11  JobLevel                  1100 non-null   int64 
 12  JobRole                   1100 non-null   object
 13  JobSatisfaction           1100 non-null   int64 
 14  MaritalStatus           

In [3]:
train.describe()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,...,1100.000000,1100.0,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,36.999091,0.161818,9.427273,2.922727,1028.157273,2.725455,2.730909,2.054545,2.732727,6483.620909,...,2.696364,80.0,0.788182,11.221818,2.807273,2.746364,7.011818,4.207273,2.226364,4.123636
std,9.037230,0.368451,8.196694,1.022242,598.915204,1.098053,0.706366,1.107805,1.109731,4715.293419,...,1.095356,0.0,0.843347,7.825548,1.291514,0.701121,6.223093,3.618115,3.313830,3.597996
min,18.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,2.000000,2.000000,504.250000,2.000000,2.000000,1.000000,2.000000,2924.500000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,0.000000,7.000000,3.000000,1026.500000,3.000000,3.000000,2.000000,3.000000,4857.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,0.000000,15.000000,4.000000,1556.500000,4.000000,3.000000,3.000000,4.000000,8354.500000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1.000000,29.000000,5.000000,2065.000000,4.000000,4.000000,5.000000,4.000000,19999.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,37.000000,18.000000,15.000000,17.000000


In [4]:
test.describe()

,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.0,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,36.471429,8.391429,2.868571,1023.285714,2.714286,2.734286,2.068571,2.725714,6479.491429,2.691429,...,2.745714,80.0,0.817143,11.202857,2.782857,2.808571,6.782857,4.260000,1.951429,4.017143
std,9.373378,7.685318,1.029583,612.566819,1.067129,0.726669,1.089615,1.083437,4633.609813,2.456892,...,1.041226,0.0,0.886539,7.470399,1.295238,0.722488,5.489113,3.622336,2.752532,3.383720
min,18.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1051.000000,0.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,463.250000,2.000000,2.000000,1.000000,2.000000,2888.750000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,35.000000,7.000000,3.000000,1011.000000,3.000000,3.000000,2.000000,3.000000,5104.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,42.000000,11.000000,4.000000,1584.500000,4.000000,3.000000,3.000000,4.000000,8260.250000,4.000000,...,4.000000,80.0,1.000000,15.750000,3.000000,3.000000,9.750000,7.000000,2.000000,7.000000
max,60.000000,29.000000,5.000000,2068.000000,4.000000,4.000000,5.000000,4.000000,19973.000000,9.000000,...,4.000000,80.0,3.000000,37.000000,6.000000,4.000000,29.000000,16.000000,15.000000,14.000000


In [5]:
train.corr()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.175393,0.007081,0.198558,-0.010953,0.011803,0.066528,0.513882,-0.003744,0.500163,...,0.063489,NaN,-0.002413,0.682879,-0.051702,-0.001042,0.328651,0.231842,0.230587,0.212540
Attrition,-0.175393,1.000000,0.088563,-0.046494,-0.045168,-0.097003,-0.122722,-0.168775,-0.125568,-0.155521,...,-0.051749,NaN,-0.138498,-0.187922,-0.043395,-0.048794,-0.143697,-0.163059,-0.071760,-0.158558
DistanceFromHome,0.007081,0.088563,1.000000,0.011437,0.029930,-0.010308,0.012333,0.016470,-0.009641,-0.017757,...,0.018112,NaN,0.050356,0.001287,-0.041208,-0.050950,0.000044,0.019317,-0.002760,0.008852
Education,0.198558,-0.046494,0.011437,1.000000,0.055979,-0.032698,0.022843,0.084075,-0.010201,0.079834,...,-0.006346,NaN,0.035881,0.125672,-0.021629,0.003099,0.074522,0.064363,0.067754,0.071870
EmployeeNumber,-0.010953,-0.045168,0.029930,0.055979,1.000000,0.030428,-0.008589,-0.006121,-0.042443,-0.007147,...,-0.073687,NaN,0.049967,-0.009526,0.017175,0.000548,-0.011106,-0.005649,-0.019621,-0.005333
EnvironmentSatisfaction,0.011803,-0.097003,-0.010308,-0.032698,0.030428,1.000000,-0.028467,-0.015355,0.000212,-0.026410,...,0.033515,NaN,0.008874,-0.018532,-0.045686,0.026477,-0.012574,0.003572,0.008843,-0.020190
JobInvolvement,0.066528,-0.122722,0.012333,0.022843,-0.008589,-0.028467,1.000000,0.005983,-0.016382,0.006114,...,0.048363,NaN,0.029483,0.018380,-0.018001,-0.025862,-0.032189,0.001194,-0.031097,0.014176
JobLevel,0.513882,-0.168775,0.016470,0.084075,-0.006121,-0.015355,0.005983,1.000000,-0.005894,0.950776,...,0.042156,NaN,0.002638,0.784020,-0.034620,0.041258,0.544091,0.411481,0.395195,0.376119
JobSatisfaction,-0.003744,-0.125568,-0.009641,-0.010201,-0.042443,0.000212,-0.016382,-0.005894,1.000000,-0.009752,...,-0.033138,NaN,0.021123,-0.023343,0.002754,-0.042767,-0.013772,-0.011798,-0.009761,-0.041852
MonthlyIncome,0.500163,-0.155521,-0.017757,0.079834,-0.007147,-0.026410,0.006114,0.950776,-0.009752,1.000000,...,0.050121,NaN,-0.006320,0.772043,-0.039713,0.032247,0.520641,0.388163,0.386875,0.344237


In [6]:
train.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,Travel_Rarely,Research & Development,1,4,Life Sciences,77,1,Male,...,3,80,1,7,2,4,7,5,0,7
1,54,0,Travel_Frequently,Research & Development,1,4,Life Sciences,1245,4,Female,...,1,80,1,33,2,1,5,4,1,4
2,34,1,Travel_Frequently,Research & Development,7,3,Life Sciences,147,1,Male,...,4,80,0,9,3,3,9,7,0,6
3,39,0,Travel_Rarely,Research & Development,1,1,Life Sciences,1026,4,Female,...,3,80,1,21,3,3,21,6,11,8
4,28,1,Travel_Frequently,Research & Development,1,3,Medical,1111,1,Male,...,1,80,2,1,2,3,1,0,0,0


In [7]:
test["Attrition"] = -1
print(len(test.columns))

31


In [8]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train, test]).reset_index(drop=True)
print(data.head())

   Age  Attrition     BusinessTravel              Department  \
0   37          0      Travel_Rarely  Research & Development   
1   54          0  Travel_Frequently  Research & Development   
2   34          1  Travel_Frequently  Research & Development   
3   39          0      Travel_Rarely  Research & Development   
4   28          1  Travel_Frequently  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeNumber  \
0                 1          4  Life Sciences              77   
1                 1          4  Life Sciences            1245   
2                 7          3  Life Sciences             147   
3                 1          1  Life Sciences            1026   
4                 1          3        Medical            1111   

   EnvironmentSatisfaction  Gender  ...  RelationshipSatisfaction  \
0                        1    Male  ...                         3   
1                        4  Female  ...                         1   
2                

In [9]:
path='./'

import os
import json
import math

from numba import jit

from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model

import lightgbm as lgb
import catboost as cbt
import xgboost as xgb


import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

# Garbage collection
import gc

# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist

from collections import Counter 
from statistics import mode 

# Warning
import warnings
warnings.filterwarnings("ignore")

from itertools import product
import ast 

### LabelEncoder处理类别特征

In [10]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ["Attrition"]] # 类别列
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    #  data['count_' + i] = data.groupby([i])[i].transform('count')  增加count特征
    data[i] = lbl.fit_transform(data[i].astype(str))

In [11]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,...,3,80,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,...,1,80,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,...,4,80,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,...,3,80,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,...,1,80,2,1,2,3,1,0,0,0


In [12]:
# 特征
feats = [i for i in data.columns if i not in ["Attrition"]]
feats

['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

### 模型训练和预测

In [13]:
model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    num_leaves=70,
    reg_alpha=0,
    reg_lambda=0,
    max_depth=-1,
    n_estimators=1500,
    objective="binary", # 二分类
    metric="F1", # 评估 F1 score
    subsample=0.95,
    colsample_bytree=0.95,
    subsample_freq=1,
    learning_rate=0.002,
    random_state=2017
)

train_x = data[data["Attrition"] != -1][feats]
train_y = data[data["Attrition"] != -1]["Attrition"]
test_x = data[data["Attrition"] == -1][feats]

# 看情况使用train_test_split切割数据集

# 训练
model.fit(train_x, train_y)

# 预测
test_pre = model.predict_proba(test_x)[:,1]
# print(test_pre[:20])

### 5折交叉验证

In [14]:
data.Attrition.value_counts()

 0    922
-1    350
 1    178
Name: Attrition, dtype: int64

In [15]:
### 5折交叉验证
from sklearn.model_selection import KFold

n_splits = 5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
data.loc[data["Attrition"] == 2, "Attrition"] = 0
train_x = data[data["Attrition"] != -1][feats]
train_y = data[data["Attrition"] != -1]["Attrition"]
res = data[data["Attrition"] == -1][["Attrition"]]
# print(res[:10])

test_x = data[data["Attrition"] == -1][feats]
res["pred"] = 0 # 预测列设置为0

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    
    model.fit(train_x1, train_y1, eval_set = [(train_x1, train_y1), (test_x1, test_y1)], eval_metric="auc", early_stopping_rounds=100)
    res["pred"] += model.predict_proba(test_x)[:,1]
    
res["pred"] = res["pred"] / 5

[1]	training's auc: 0.88128	valid_1's auc: 0.724317
[2]	training's auc: 0.91224	valid_1's auc: 0.712381
[3]	training's auc: 0.909679	valid_1's auc: 0.705778
[4]	training's auc: 0.919316	valid_1's auc: 0.75073
[5]	training's auc: 0.923066	valid_1's auc: 0.747619
[6]	training's auc: 0.924002	valid_1's auc: 0.753778
[7]	training's auc: 0.92374	valid_1's auc: 0.756317
[8]	training's auc: 0.923025	valid_1's auc: 0.754667
[9]	training's auc: 0.923982	valid_1's auc: 0.750984
[10]	training's auc: 0.927112	valid_1's auc: 0.75454
[11]	training's auc: 0.925698	valid_1's auc: 0.751873
[12]	training's auc: 0.925215	valid_1's auc: 0.752635
[13]	training's auc: 0.92531	valid_1's auc: 0.753905
[14]	training's auc: 0.926458	valid_1's auc: 0.752254
[15]	training's auc: 0.928481	valid_1's auc: 0.751492
[16]	training's auc: 0.928984	valid_1's auc: 0.75073
[17]	training's auc: 0.930021	valid_1's auc: 0.752
[18]	training's auc: 0.930373	valid_1's auc: 0.751492
[19]	training's auc: 0.930746	valid_1's auc: 0.

[48]	training's auc: 0.925088	valid_1's auc: 0.851449
[49]	training's auc: 0.925908	valid_1's auc: 0.851902
[50]	training's auc: 0.926767	valid_1's auc: 0.851902
[51]	training's auc: 0.926729	valid_1's auc: 0.851902
[52]	training's auc: 0.92692	valid_1's auc: 0.851298
[53]	training's auc: 0.926815	valid_1's auc: 0.851298
[54]	training's auc: 0.927464	valid_1's auc: 0.849638
[55]	training's auc: 0.928447	valid_1's auc: 0.849034
[56]	training's auc: 0.928294	valid_1's auc: 0.849336
[57]	training's auc: 0.92796	valid_1's auc: 0.849487
[58]	training's auc: 0.928141	valid_1's auc: 0.850091
[59]	training's auc: 0.928561	valid_1's auc: 0.852053
[60]	training's auc: 0.92838	valid_1's auc: 0.8516
[61]	training's auc: 0.928867	valid_1's auc: 0.850393
[62]	training's auc: 0.928943	valid_1's auc: 0.850694
[63]	training's auc: 0.929143	valid_1's auc: 0.850845
[64]	training's auc: 0.928943	valid_1's auc: 0.849638
[65]	training's auc: 0.929487	valid_1's auc: 0.848732
[66]	training's auc: 0.929659	val

[199]	training's auc: 0.9487	valid_1's auc: 0.856733
[200]	training's auc: 0.948853	valid_1's auc: 0.856733
[201]	training's auc: 0.948872	valid_1's auc: 0.857186
[202]	training's auc: 0.949063	valid_1's auc: 0.857639
[203]	training's auc: 0.949254	valid_1's auc: 0.857488
[204]	training's auc: 0.94933	valid_1's auc: 0.857186
[205]	training's auc: 0.949693	valid_1's auc: 0.85628
[206]	training's auc: 0.949721	valid_1's auc: 0.856884
[207]	training's auc: 0.949845	valid_1's auc: 0.857035
[208]	training's auc: 0.950017	valid_1's auc: 0.855676
[209]	training's auc: 0.950084	valid_1's auc: 0.855072
[210]	training's auc: 0.950208	valid_1's auc: 0.855223
[211]	training's auc: 0.950246	valid_1's auc: 0.855374
[212]	training's auc: 0.950427	valid_1's auc: 0.855072
[213]	training's auc: 0.950447	valid_1's auc: 0.855072
[214]	training's auc: 0.950714	valid_1's auc: 0.855827
[215]	training's auc: 0.950952	valid_1's auc: 0.856582
[216]	training's auc: 0.950914	valid_1's auc: 0.856582
[217]	training

[353]	training's auc: 0.963739	valid_1's auc: 0.863074
[354]	training's auc: 0.963806	valid_1's auc: 0.862772
[355]	training's auc: 0.96393	valid_1's auc: 0.862621
[356]	training's auc: 0.964111	valid_1's auc: 0.862319
[357]	training's auc: 0.964168	valid_1's auc: 0.862168
[358]	training's auc: 0.964254	valid_1's auc: 0.862319
[359]	training's auc: 0.964369	valid_1's auc: 0.862168
[360]	training's auc: 0.964445	valid_1's auc: 0.861111
[361]	training's auc: 0.96455	valid_1's auc: 0.860507
[362]	training's auc: 0.96455	valid_1's auc: 0.860658
[363]	training's auc: 0.964588	valid_1's auc: 0.859903
[364]	training's auc: 0.964579	valid_1's auc: 0.859601
[365]	training's auc: 0.964674	valid_1's auc: 0.858998
[366]	training's auc: 0.964798	valid_1's auc: 0.860054
[367]	training's auc: 0.964808	valid_1's auc: 0.859752
[368]	training's auc: 0.964932	valid_1's auc: 0.859752
[369]	training's auc: 0.964932	valid_1's auc: 0.859601
[370]	training's auc: 0.964941	valid_1's auc: 0.860054
[371]	trainin

[504]	training's auc: 0.974331	valid_1's auc: 0.865187
[505]	training's auc: 0.974379	valid_1's auc: 0.86564
[506]	training's auc: 0.974427	valid_1's auc: 0.866093
[507]	training's auc: 0.974512	valid_1's auc: 0.866395
[508]	training's auc: 0.974493	valid_1's auc: 0.866546
[509]	training's auc: 0.974665	valid_1's auc: 0.866395
[510]	training's auc: 0.974684	valid_1's auc: 0.866244
[511]	training's auc: 0.974722	valid_1's auc: 0.866395
[512]	training's auc: 0.974875	valid_1's auc: 0.866546
[513]	training's auc: 0.974904	valid_1's auc: 0.866848
[514]	training's auc: 0.974999	valid_1's auc: 0.866546
[515]	training's auc: 0.975066	valid_1's auc: 0.866848
[516]	training's auc: 0.975142	valid_1's auc: 0.86715
[517]	training's auc: 0.975228	valid_1's auc: 0.866697
[518]	training's auc: 0.975238	valid_1's auc: 0.866546
[519]	training's auc: 0.975409	valid_1's auc: 0.866697
[520]	training's auc: 0.975371	valid_1's auc: 0.866546
[521]	training's auc: 0.975457	valid_1's auc: 0.866848
[522]	traini

[19]	training's auc: 0.918483	valid_1's auc: 0.72794
[20]	training's auc: 0.918975	valid_1's auc: 0.727343
[21]	training's auc: 0.918892	valid_1's auc: 0.728708
[22]	training's auc: 0.919059	valid_1's auc: 0.727684
[23]	training's auc: 0.919532	valid_1's auc: 0.725806
[24]	training's auc: 0.919059	valid_1's auc: 0.72666
[25]	training's auc: 0.918864	valid_1's auc: 0.728367
[26]	training's auc: 0.918799	valid_1's auc: 0.730244
[27]	training's auc: 0.918771	valid_1's auc: 0.728025
[28]	training's auc: 0.918706	valid_1's auc: 0.732634
[29]	training's auc: 0.918539	valid_1's auc: 0.733487
[30]	training's auc: 0.918474	valid_1's auc: 0.734682
[31]	training's auc: 0.919012	valid_1's auc: 0.732122
[32]	training's auc: 0.919272	valid_1's auc: 0.731439
[33]	training's auc: 0.919356	valid_1's auc: 0.730415
[34]	training's auc: 0.919838	valid_1's auc: 0.729391
[35]	training's auc: 0.920043	valid_1's auc: 0.728025
[36]	training's auc: 0.920265	valid_1's auc: 0.727684
[37]	training's auc: 0.920182	

[122]	training's auc: 0.934814	valid_1's auc: 0.742632
[123]	training's auc: 0.934906	valid_1's auc: 0.741579
[124]	training's auc: 0.934925	valid_1's auc: 0.742632
[125]	training's auc: 0.934962	valid_1's auc: 0.742632
[126]	training's auc: 0.935155	valid_1's auc: 0.742456
[127]	training's auc: 0.935451	valid_1's auc: 0.742632
[128]	training's auc: 0.9357	valid_1's auc: 0.743333
[129]	training's auc: 0.935765	valid_1's auc: 0.743684
[130]	training's auc: 0.935922	valid_1's auc: 0.743684
[131]	training's auc: 0.936198	valid_1's auc: 0.743684
[132]	training's auc: 0.936309	valid_1's auc: 0.742807
[133]	training's auc: 0.9363	valid_1's auc: 0.742105
[134]	training's auc: 0.936355	valid_1's auc: 0.743684
[135]	training's auc: 0.936448	valid_1's auc: 0.743158
[136]	training's auc: 0.936651	valid_1's auc: 0.743158
[137]	training's auc: 0.936863	valid_1's auc: 0.743333
[138]	training's auc: 0.93738	valid_1's auc: 0.745614
[139]	training's auc: 0.937408	valid_1's auc: 0.745614
[140]	training'

[331]	training's auc: 0.955943	valid_1's auc: 0.761754
[332]	training's auc: 0.956035	valid_1's auc: 0.761754
[333]	training's auc: 0.956035	valid_1's auc: 0.762105
[334]	training's auc: 0.956063	valid_1's auc: 0.761228
[335]	training's auc: 0.956146	valid_1's auc: 0.761754
[336]	training's auc: 0.956321	valid_1's auc: 0.762281
[337]	training's auc: 0.956469	valid_1's auc: 0.762456
[338]	training's auc: 0.95657	valid_1's auc: 0.762105
[339]	training's auc: 0.956635	valid_1's auc: 0.761579
[340]	training's auc: 0.956709	valid_1's auc: 0.762281
[341]	training's auc: 0.956866	valid_1's auc: 0.762456
[342]	training's auc: 0.956856	valid_1's auc: 0.762281
[343]	training's auc: 0.95694	valid_1's auc: 0.762281
[344]	training's auc: 0.956995	valid_1's auc: 0.762456
[345]	training's auc: 0.957115	valid_1's auc: 0.762982
[346]	training's auc: 0.957133	valid_1's auc: 0.762456
[347]	training's auc: 0.957327	valid_1's auc: 0.763684
[348]	training's auc: 0.957401	valid_1's auc: 0.763333
[349]	traini

[539]	training's auc: 0.972031	valid_1's auc: 0.779298
[540]	training's auc: 0.972115	valid_1's auc: 0.78
[541]	training's auc: 0.972216	valid_1's auc: 0.780351
[542]	training's auc: 0.97229	valid_1's auc: 0.780175
[543]	training's auc: 0.972419	valid_1's auc: 0.78
[544]	training's auc: 0.972475	valid_1's auc: 0.780175
[545]	training's auc: 0.972521	valid_1's auc: 0.780175
[546]	training's auc: 0.972585	valid_1's auc: 0.780702
[547]	training's auc: 0.972622	valid_1's auc: 0.780702
[548]	training's auc: 0.972733	valid_1's auc: 0.781228
[549]	training's auc: 0.972742	valid_1's auc: 0.781228
[550]	training's auc: 0.972835	valid_1's auc: 0.781228
[551]	training's auc: 0.972955	valid_1's auc: 0.781579
[552]	training's auc: 0.973019	valid_1's auc: 0.781579
[553]	training's auc: 0.973056	valid_1's auc: 0.781228
[554]	training's auc: 0.973102	valid_1's auc: 0.781579
[555]	training's auc: 0.973268	valid_1's auc: 0.781228
[556]	training's auc: 0.973435	valid_1's auc: 0.781404
[557]	training's au

In [16]:
pd.DataFrame({"result":res["pred"]}).to_csv(path + "output/lgb_proba.csv", index=False)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()

testX = test_x.values
trainY = train_y.values
trainX = train_x.values

model.fit(trainX, trainY)
predictY = model.predict_proba(testX)[:,1]

In [18]:
pd.DataFrame({"result": predictY}).to_csv(path + "output/gbdt_proba.csv", index=False)

In [19]:
res.loc[res["pred"] >= 0.5, "pred"] = 1
res.loc[res["pred"] < 0.5, "pred"] = 0
# res.columns = ["Attrition", "result"]
res.columns

Index(['Attrition', 'pred'], dtype='object')

In [22]:
res["result"] = res["pred"].astype(int)
res[["result"]].to_csv(path + "output/lgb_submission_5_fold.csv", index=False)

res[["result"]]
# res["result"]

,result
1100,0
1101,0
1102,0
1103,0
1104,0
...,...
1445,0
1446,0
1447,0
1448,0
